<a href="https://colab.research.google.com/github/NagaGun/F1-Auto/blob/main/F1_Auto_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

'''
def part1():
    # Tensor addition and subtraction
    a = torch.tensor([1, 2, 3])
    b = torch.tensor([4, 5, 6])
    print("Addition:", a + b)
    # Element-wise addition
    print("Subtraction:", a - b)
    # Element-wise subtraction


def part2():
    # Tensor multiplication
    a = torch.tensor([[1, 2], [3, 4]])
    b = torch.tensor([[5, 6], [7, 8]])
    print("Element-wise Multiplication:\n", a * b)
    # Element-wise multiplication
    print("Matrix Multiplication:\n", torch.matmul(a, b))
    # Matrix multiplication


def part3():
    # Squeezing, unsqueezing, and reshaping
    a = torch.tensor([[[1, 2, 3]]])  # 3D tensor
    print("Original Tensor Shape:", a.shape)
    b = a.squeeze()  # Removes dimensions of size 1
    print("Squeezed Tensor Shape:", b.shape)
    c = b.unsqueeze(0)  # Adds a dimension at position 0
    print("Unsqueezed Tensor Shape:", c.shape)
    d = c.reshape(3, 1)  # Reshapes to 3x1
    print("Reshaped Tensor:\n", d)

if __name__ == "__main__":
    print(torch.__version__)
    part1()
    part2()
    part3()
'''
### How to predict prime numbers###
def is_prime(n):
    if n < 2:
        return 0
    for i in range(2, int(n ** 0.5) + 1):
        if n % i == 0:
            return 0
    return 1

# Create dataset
numbers = np.arange(1, 1001)
labels = np.array([is_prime(n) for n in numbers])

# Features: n, n%2, n%3, n%5, n%7
features = np.stack([
    numbers / 1000,       # normalized number
    numbers % 2,
    numbers % 3,
    numbers % 5,
    numbers % 7
], axis=1)

X = torch.tensor(features, dtype=torch.float32)
y = torch.tensor(labels, dtype=torch.float32).unsqueeze(1)

class PrimePredictor(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(5, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        return self.sigmoid(self.fc3(x))

model = PrimePredictor()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

for epoch in range(500):
    outputs = model(X)
    loss = criterion(outputs, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if (epoch+1) % 100 == 0:
        print(f"Epoch [{epoch+1}/500], Loss: {loss.item():.4f}")

with torch.no_grad():
    test_nums = np.array([2, 4, 17, 20, 997])
    test_features = np.stack([
        test_nums / 1000,
        test_nums % 2,
        test_nums % 3,
        test_nums % 5,
        test_nums % 7
    ], axis=1)
    test_tensors = torch.tensor(test_features, dtype=torch.float32)
    preds = model(test_tensors)
    print("Predictions:", preds.squeeze().numpy())
    print("Rounded:", (preds > 0.5).int().squeeze().numpy())

Epoch [100/500], Loss: 0.1368
Epoch [200/500], Loss: 0.1284
Epoch [300/500], Loss: 0.1228
Epoch [400/500], Loss: 0.1166
Epoch [500/500], Loss: 0.1091
Predictions: [8.2489818e-01 1.3096917e-07 8.7380075e-01 5.2034665e-10 5.1380891e-01]
Rounded: [1 0 1 0 1]


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving 000000000139.jpg to 000000000139.jpg
Saving 000000000285.jpg to 000000000285.jpg
Saving 000000000632.jpg to 000000000632.jpg
Saving 000000000724.jpg to 000000000724.jpg
Saving 000000000776.jpg to 000000000776.jpg
Saving 000000000785.jpg to 000000000785.jpg
Saving 000000000802.jpg to 000000000802.jpg
Saving 000000000872.jpg to 000000000872.jpg
Saving 000000000885.jpg to 000000000885.jpg
Saving 000000001000.jpg to 000000001000.jpg
Saving 000000001532.jpg to 000000001532.jpg
Saving 000000001584.jpg to 000000001584.jpg
Saving 000000001675.jpg to 000000001675.jpg
Saving 000000001761.jpg to 000000001761.jpg
Saving 000000001818.jpg to 000000001818.jpg
Saving 000000001993.jpg to 000000001993.jpg
Saving 000000002006.jpg to 000000002006.jpg
Saving 000000002149.jpg to 000000002149.jpg
Saving 000000002153.jpg to 000000002153.jpg
Saving 000000002157.jpg to 000000002157.jpg
Saving 000000002592.jpg to 000000002592.jpg
Saving 000000002685.jpg to 000000002685.jpg
Saving 000000002923.jpg to 00000

In [ ]:
!pip install torch torchvision torchaudio matplotlib

import torch
from torchvision.models.detection import fasterrcnn_resnet50_fpn, FasterRCNN_ResNet50_FPN_Weights
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
import os

folder_path = "/content"
image_files = [f for f in os.listdir(folder_path) if f.endswith(".jpg") or f.endswith(".png")]
print(f"Found {len(image_files)} images: {image_files}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

weights = FasterRCNN_ResNet50_FPN_Weights.DEFAULT
model = fasterrcnn_resnet50_fpn(weights=weights)
model.eval()
model.to(device)

transform = transforms.Compose([
    transforms.ToTensor()
])


cone_category_id = 81  # COCO ID for traffic cone


for img_file in image_files:
    img_path = os.path.join(folder_path, img_file)
    image = Image.open(img_path).convert("RGB")
    img_tensor = transform(image).to(device)

    with torch.no_grad():
        prediction = model([img_tensor])

    boxes = prediction[0]['boxes']
    labels = prediction[0]['labels']
    scores = prediction[0]['scores']

    cone_boxes = boxes[(labels == cone_category_id) & (scores > 0.5)]


    plt.figure(figsize=(8, 8))
    plt.imshow(image)
    ax = plt.gca()

    for box in cone_boxes:
        x1, y1, x2, y2 = box.cpu().numpy()
        ax.add_patch(plt.Rectangle((x1, y1), x2-x1, y2-y1, fill=False, color='red', linewidth=2))

    plt.title(f"{img_file} - Cones Detected: {len(cone_boxes)}")
    plt.axis('off')
    plt.show()

Found 0 images: []
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


100%|██████████| 160M/160M [00:01<00:00, 158MB/s]
